In [25]:
import logging
import os
import sys
from spleeter.separator import Separator
from spleeter.audio.adapter import AudioAdapter
import soundfile as sf
import numpy as np
import librosa
from PIL import Image, ImageOps
from pathlib import Path
import pydub

import cv2

from riffusion.spectrogram_image_converter import SpectrogramImageConverter
from riffusion.spectrogram_converter import SpectrogramConverter
from riffusion.spectrogram_params import SpectrogramParams

sample_audio_path = "raw-audio/blues.00000.wav"

In [26]:
img_converter = SpectrogramImageConverter(SpectrogramParams(min_frequency=0, max_frequency=10000))

c:\Users\18nee\Desktop\stanford\wi22\EE269\mel-train\riffusion\spectrogram_image_converter.py:21: UserWarning: WARNING: cuda is not available, using cpu instead.
  self.converter = SpectrogramConverter(params=params, device=device)


In [31]:
audio_loader = AudioAdapter.default()
waveform, _ = audio_loader.load(sample_audio_path, sample_rate=44100)
print(waveform, type(waveform))
song = pydub.AudioSegment(
    waveform.tobytes(), 
    frame_rate=44100,
    sample_width=waveform.dtype.itemsize, 
    channels=1
)

out_img = img_converter.spectrogram_image_from_audio(song)
out_img.save("temp_spec.png")

[[ 0.00732422]
 [ 0.01099474]
 [ 0.01660156]
 ...
 [-0.06466275]
 [-0.06417847]
 [-0.06310088]] <class 'numpy.ndarray'>


In [29]:
recon_filepath = "temp_spec.png"

recon_img =  Image.open(recon_filepath).convert("RGB")

img_converter_to_audio = SpectrogramImageConverter(SpectrogramParams(sample_rate=44100, min_frequency=0, max_frequency=10000))

out_audio_recon = img_converter_to_audio.audio_from_spectrogram_image(recon_img, apply_filters=True).set_channels(1)
out_audio_recon.export("temp_wav.wav", format="wav")

<_io.BufferedRandom name='temp_wav.wav'>